# Notifications Matrix Generator

This application provides a user-friendly interface to generate notification strategies based on various parameters like app category, product features, special app features, target audience, and tone of voice. It leverages the power of OpenAI's GPT model and the PromptLayer API to generate these strategies.

## Import necessary libraries and modules

In [ ]:
from dotenv import load_dotenv
import gradio as gr
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import PromptLayerChatOpenAI
from langchain.output_parsers import PydanticOutputParser
import os
import promptlayer
from pydantic import BaseModel, Field, validator
from typing import List

## Load environment variables

In [ ]:
load_dotenv()

## Set API Keys

In [ ]:
promptlayer.api_key = os.environ.get("PROMPTLAYER_API_KEY")
openai = promptlayer.openai
openai.api_key = os.environ.get("OPENAI_API_KEY")

## Define GPT model and PromptLayer Tag

In [ ]:
GPT_MODEL = "gpt-4"
TEMPERATURE = 0.7
PROMPTLAYER_TAG="NotificationsMatrixGenerator"

## Define the Desired Data Structure Output

In [ ]:
class Notification(BaseModel):
    User_Segment: str = Field(description="the user segment for the notification")
    Business_Goal: str = Field(description="the business goal associated with the user segment")
    Notification_Type: str = Field(description="the type of notification to be sent")
    Timing_Rule: str = Field(description="the rule for when the notification should be sent")
    Generated_Message: str = Field(description="the message to be sent in the notification")
    Justification: str = Field(description="the reason for sending the notification")

class List_of_Notifications(BaseModel):
    notifications: List[Notification]

## Set up the parser + inject instructions into the prompt template

In [ ]:
parser = PydanticOutputParser(pydantic_object=List_of_Notifications)
format_instructions = parser.get_format_instructions()

## Generate LLM Response from Template

In [ ]:
def generate_llm_response_from_template(template, **kwargs):
    prompt_template = PromptTemplate(
        input_variables=list(kwargs.keys()),
        template=template,
        partial_variables={"format_instructions": format_instructions}
    )
    llm = PromptLayerChatOpenAI(model=GPT_MODEL, pl_tags=[PROMPTLAYER_TAG])
    llm_chain = LLMChain(llm=llm, prompt=prompt_template)
    response = llm_chain.predict(**kwargs)
    return response

## Define Prompt Template

In [ ]:
prompt_template="""As an expert in mobile app notification strategies and adhering to OneSignal best practices:

Consider the following app information:
- Category: {app_category}
- Special Features: {special_features}
- Product Features: {product_features}
- Target Audience: {target_audience}
- Tone of Voice: {tone_of_voice}

Considering industry best practices and OneSignal guidelines, provide a comprehensive list of notifications with each item containing the following information:
- User Segment
- Business Goal
- Notification Type
- Timing Rule
- Generated Message
- Justification

format_instructions: {format_instructions}
"""

## Get Input parameters to run the model

Example inputs:
1. **App Category**: Education
2. **Product Features**: Vocabulary Lessons, Grammar Tips, Practice Exercises, Daily Quizzes
3. **Special App Features**: Subscriptions, Learning Tracks/Courses, Daily/Weekly Challenges
4. **Target Audience**: Adults
5. **Tone of Voice**: Friendly

In [ ]:
app_category = input("Enter the App Category: ").strip()
special_features = input("Enter the Special Features: ").strip()
product_features = input("Enter the Product Features: ").strip()
target_audience = input("Enter the Target Audience: ").strip()
tone_of_voice = input("Enter the Tone of Voice: ").strip()

## Generate response based on inputs

In [ ]:
response = generate_llm_response_from_template(template=prompt_template, app_category=app_category, special_features=special_features, product_features=product_features, target_audience=target_audience, tone_of_voice=tone_of_voice)
print(response)